<a href="https://colab.research.google.com/github/StaniszewskiA/Loss-Functions/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from typing import Dict
import torch
from torch import optim
from torchvision import datasets, transforms


def get_config_cnnh_cifar():
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        device = torch.device("cpu")

    config = {
        "T": 30,
        "hash_save_path": "save/CNNH/",
        "optimizer": {
            "type": optim.Adam,
            "optim_params": {
                "lr": 1e-4,
                "betas": (0.9, 0.999)
            }
        },
        "info": "[CNNH]",
        "resize_size": 64,
        "crop_size": 32,
        "batch_size": 8,
        "net": "ResNet",
        "dataset": "cifar",
        "epochs": 100,
        "save_interval": 10,
        "test_MAP": 10,
        "device": device,
        "bit_list": [12],
    }

    return config

cnnh_config_cifar = get_config_cnnh_cifar()

transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(32),
    transforms.ToTensor(),
])

cifar_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(cifar_dataset, batch_size=8, shuffle=True, num_workers=4)




Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [35]:
def calculate_total_loss(hash_matrix, sim_matrix, anchor, anchor_positive_similarity, anchor_negative_similarity):
  reconstruction_loss = torch.nn.functional.mse_loss(hash_matrix, sim_matrix)
  #Triplet loss
  margin = 1.0
  anchors_tensor = torch.from_numpy(anchors)
  positives_tensor = torch.from_numpy(positives)
  negatives_tensor = torch.from_numpy(negatives)
  triplet_loss = torch.nn.functional.triplet_margin_loss(anchors_tensor, positives_tensor, negatives_tensor, margin=margin)

  alpha = 0.3
  loss = alpha * reconstruction_loss + (1 - alpha) * triplet_loss

  print(loss)


hashes = 2 * torch.rand((8, 8)) - 1
hash_matrix = hashes @ hashes.t()
sim_matrix = 2 * torch.rand((8, 8)) - 1

positive_indices = torch.randint(0, hashes.size(0), (8,))
negative_indices = torch.randint(0, hashes.size(0), (8,))



In [5]:
bits = 12
n = 10000
train_labels = torch.tensor([], dtype=torch.long)

for data, labels in train_loader:
    train_labels = torch.cat((train_labels, labels), dim=0)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
hashes = 2 * torch.rand((n, bits)) - 1

In [13]:
import numpy as np

train_labels_capped = train_labels[:10000]
sim_matrix = np.outer(train_labels_capped, train_labels_capped)
hash_numpy = np.array(hashes)
hash_transposed = hash_numpy.T

hash_matrix = np.dot(hash_numpy, hash_transposed)
print(hash_matrix)

[[ 2.7619655   0.7458481  -1.1143789  ...  0.5674983   0.933864
   0.18928388]
 [ 0.7458481   5.979451   -0.8006763  ... -1.131736   -0.15816414
  -1.1609393 ]
 [-1.1143789  -0.8006763   2.9577866  ... -0.16948736  0.29779795
  -0.3639501 ]
 ...
 [ 0.5674983  -1.131736   -0.16948736 ...  2.5785925  -1.2006694
   1.1310997 ]
 [ 0.933864   -0.15816414  0.29779795 ... -1.2006694   3.6475048
   0.21115881]
 [ 0.18928388 -1.1609393  -0.3639501  ...  1.1310997   0.21115881
   4.1738977 ]]


In [19]:
def generate_triplets(labels, hashes):
  triplets = []
  unique_labels = np.unique(labels)

  for label in unique_labels:
    label_indices = np.where(labels == label)[0]

    if len(label_indices) < 2:
            # Skip if there are less than two samples for the current label
            continue

    anchor_pos_pairs = np.random.choice(label_indices, size=(len(label_indices), 2), replace=True)

    for anchor_idx, pos_idx in anchor_pos_pairs:
      negative_label = label
      while negative_label == label:
          negative_label = np.random.choice(unique_labels)

      negative_indices = np.where(labels == negative_label)[0]
      neg_idx = np.random.choice(negative_indices)

      # Append the triplet [anchor, positive, negative] to the list
      triplet = (hashes[anchor_idx], hashes[pos_idx], hashes[neg_idx])
      triplets.append(triplet)

    return np.array(triplets)

In [24]:
triplets = generate_triplets(train_labels[:10000], hash_numpy)
for i in range(5):
    print("Triplet {}: Anchor - {}, Positive - {}, Negative - {}".format(i+1, triplets[i][0], triplets[i][1], triplets[i][2]))

Triplet 1: Anchor - [-0.78022134 -0.15015352 -0.67671406  0.20092511  0.85447574 -0.4039936
  0.35563433 -0.77577865  0.16380537 -0.68803287 -0.6849495  -0.65226984], Positive - [ 0.85655713 -0.06814587  0.33272076 -0.5474824  -0.24060702  0.6803856
 -0.5858886  -0.5691786   0.855338    0.36310172 -0.5310979   0.39090657], Negative - [-0.28000426 -0.58267117  0.04504192 -0.3847071  -0.5881288   0.9838792
  0.29649985  0.9695395   0.33961844  0.8838202  -0.9405385  -0.8587575 ]
Triplet 2: Anchor - [ 0.7398747   0.69438434 -0.04233432 -0.1617055  -0.11535013 -0.21859801
 -0.7656318  -0.09557378  0.3232448   0.43168592 -0.8978088   0.682775  ], Positive - [-0.17124736  0.6884593   0.4962902   0.3269559  -0.6382916  -0.28881764
 -0.79716706 -0.23736918 -0.60207856  0.03670335  0.05979538 -0.5493891 ], Negative - [-0.02591014 -0.4490205  -0.7568866   0.4024192   0.82380354 -0.67825747
  0.2849412  -0.6462605   0.5900525  -0.22745776  0.8979193   0.05336714]
Triplet 3: Anchor - [ 0.02920592 

In [36]:
anchors, positives, negatives = triplets[:, 0], triplets[:, 1], triplets[:, 2]
calculate_total_loss(hash_matrix, sim_matrix, anchors, positives, negatives)

tensor(1.5164)
